### 1. Interpretation and significance

Suppose that we would like to know how much families in the US are spending on recreation annually. We've estimated the following model:

$$𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒 = 873 + 0.0012 * 𝑎𝑛𝑛𝑢𝑎𝑙\_𝑖𝑛𝑐𝑜𝑚𝑒 + 0.00002 * 𝑎𝑛𝑛𝑢𝑎𝑙\_𝑖𝑛𝑐𝑜𝑚𝑒^2 − 223.57 * ℎ𝑎𝑣𝑒\_𝑘𝑖𝑑𝑠$$

expenditure is the annual spending on recreation in US dollars, annual_income is the annual income in US dollars, and have_kids is a dummy variable indicating the families with children. Interpret the estimated coefficients. What additional statistics should be given in order to make sure that your interpretations make sense statistically. Write up your answer.

<span style="color:blue">From the intercept we can see that on average an individual will spend at least 873 on recreation, which averages out to about 73/month.  <br><br>Looking at the coefficients for income and income^2, we also get a picture that expenditure grows at an exponential rate, with the more money you make.  Although the scale of the coefficient is smaller, the scale of its variable is significantly greater.  This is relatively intuitive since people with more income tend to have more disposable income to spend on recreation.  <br><br>It also makes sense that having children would decrease the amount spent on recreation given the financial burden children tend to be.  However I don't think this is a good representation of the feature in this model.  Kids should be a numeric value, and it may be even more prudent to do some feature engineering to create a new feature as a function of anual_income / n_kids, because number of kids should affect lower income families more than higher income families.</span>

### 2. Weather model
In this exercise, you'll work with the historical temperature data from the previous checkpoint. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

* First, load the dataset from the weatherinszeged table from Thinkful's database.
* Build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed. Now, estimate your model using OLS. Are the estimated coefficients statistically significant? Are the signs of the estimated coefficients in line with your previous expectations? Interpret the estimated coefficients. What are the relations between the target and the explanatory variables?
* Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Are the coefficients statistically significant? Did the signs of the estimated coefficients for humidity and windspeed change? Interpret the estimated coefficients.

In [34]:
import pandas as pd
import numpy as np
import scipy.stats as st

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

In [3]:
confile = list(pd.read_csv('../../dbcon.csv'))
postgres_db = 'weatherinszeged'
db_connection = 'postgresql://{}:{}@{}:{}/{}'.format(confile[0], confile[1], confile[2], confile[3], postgres_db)

query = f'''
SELECT *
FROM {postgres_db}
;'''

weather = pd.read_sql(query, db_connection)
display(
    weather.isna().mean(),
    weather.info(),
    weather.head(5)
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   date                 96453 non-null  datetime64[ns, UTC]
 1   summary              96453 non-null  object             
 2   preciptype           96453 non-null  object             
 3   temperature          96453 non-null  float64            
 4   apparenttemperature  96453 non-null  float64            
 5   humidity             96453 non-null  float64            
 6   windspeed            96453 non-null  float64            
 7   windbearing          96453 non-null  float64            
 8   visibility           96453 non-null  float64            
 9   loudcover            96453 non-null  float64            
 10  pressure             96453 non-null  float64            
 11  dailysummary         96453 non-null  object             
dtypes: datetime64[ns, 

date                   0.0
summary                0.0
preciptype             0.0
temperature            0.0
apparenttemperature    0.0
humidity               0.0
windspeed              0.0
windbearing            0.0
visibility             0.0
loudcover              0.0
pressure               0.0
dailysummary           0.0
dtype: float64

None

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [4]:
weather['tempdiff'] = weather['apparenttemperature'] - weather['temperature']
X = weather[['humidity', 'windspeed']]
y = weather['tempdiff']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=36)

In [25]:
prepro = ColumnTransformer(
    [('scaler_nums', StandardScaler(), X.columns)
    ], remainder='passthrough')

pipeline = Pipeline([('prepro', prepro), 
                     ('OLS', LinearRegression())])

fit_results = pipeline.fit(X_train, y_train)

pd.DataFrame(np.concatenate([
    np.array(pipeline.named_steps['prepro'].named_transformers_['scaler_nums'].inverse_transform(pipeline.named_steps['OLS'].coef_), ndmin=2), 
    np.array(pipeline.named_steps['OLS'].intercept_, ndmin=2)
                            ], axis=1),
             columns=np.concatenate([X.columns, ['Constant']]))


,humidity,windspeed,Constant
0,0.618914,5.091484,-1.07382


<span style="color:blue">The windspeed does not line up with previously expected correlation, generally I associate increased higher windspeeds with lower apparent temperature (Chicago, Detroit, SF, SD, etc.).  The other coefficients make sense. <br>Given that humidity and windspeed can not be negative, apparent temp cannot be lower than -1 since both windspeed and humidity have a positive linear relation to temp diff.</span>

### 2. houseprices

In [32]:
confile = list(pd.read_csv('../../dbcon.csv'))
postgres_db = 'houseprices'
db_connection = 'postgresql://{}:{}@{}:{}/{}'.format(confile[0], confile[1], confile[2], confile[3], postgres_db)

query = f'''
SELECT *
FROM {postgres_db}
;'''

houseprices = pd.read_sql(query, db_connection)
display(
    houseprices.isna().mean(),
    houseprices.info(),
    houseprices.head(5)
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1460 non-null   int64  
 1   mssubclass     1460 non-null   int64  
 2   mszoning       1460 non-null   object 
 3   lotfrontage    1201 non-null   float64
 4   lotarea        1460 non-null   int64  
 5   street         1460 non-null   object 
 6   alley          91 non-null     object 
 7   lotshape       1460 non-null   object 
 8   landcontour    1460 non-null   object 
 9   utilities      1460 non-null   object 
 10  lotconfig      1460 non-null   object 
 11  landslope      1460 non-null   object 
 12  neighborhood   1460 non-null   object 
 13  condition1     1460 non-null   object 
 14  condition2     1460 non-null   object 
 15  bldgtype       1460 non-null   object 
 16  housestyle     1460 non-null   object 
 17  overallqual    1460 non-null   int64  
 18  overallc

id               0.000000
mssubclass       0.000000
mszoning         0.000000
lotfrontage      0.177397
lotarea          0.000000
                   ...   
mosold           0.000000
yrsold           0.000000
saletype         0.000000
salecondition    0.000000
saleprice        0.000000
Length: 81, dtype: float64

None

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [42]:
targetvar = 'saleprice'
cols = ['mosold', 'lotfrontage', 'lotarea', 'firstflrsf', 'garagearea', 'bedroomabvgr', 'fullbath'] + [targetvar]
houseprices = houseprices[cols].dropna().reset_index(drop=True)
X = houseprices.drop(columns=[targetvar])
y = houseprices[targetvar]

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)
scaled_X = sm.add_constant(scaled_X)
scaled_X = pd.DataFrame(scaled_X,
             columns=['Const'] + list(X.columns))

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.25, random_state=36)

results = sm.OLS(y_train, X_train).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.600
Method:                 Least Squares   F-statistic:                     193.9
Date:                Sat, 30 May 2020   Prob (F-statistic):          2.40e-174
Time:                        13:04:23   Log-Likelihood:                -10999.
No. Observations:                 900   AIC:                         2.201e+04
Df Residuals:                     892   BIC:                         2.205e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Const         1.803e+05   1648.220    109.417      0.000    1.77e+05    1.84e+05
mosold        1241.1107   1656.478      0.749      0.454   -2009.938    4492.160
lotfrontage   -807.2987   2211.063     -0.365      0.715   -5146.791    3532.193
lotarea       9779.7986   3033.683      3.224      0.001    3825.811    1.57e+04
firstflrsf    2.157e+04   2147.078     10.045      0.000    1.74e+04    2.58e+04
garagearea    2.904e+04   2065.021     14.062      0.000     2.5e+04    3.31e+04
bedroomabvgr -1585.2145   1822.383     -0.870      0.385   -5161.873    1991.444
fullbath      2.314e+04   2003.021     11.553      0.000    1.92e+04    2.71e+04
==============================================================================
Omnibus:                      161.760   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3145.875
Skew:                          -0.090   Prob(JB):                         0.00
Kurtosis:                      12.157   Cond. No.                         3.07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<span style="color:blue">lotarea, firstflrsf, garagearea, and fullbath are the statistically significant coefs<br><br>mosold, lotfrontage, and bedroomavbgr are not significant</span>

In [43]:
targetvar = 'saleprice'
cols = ['lotarea', 'firstflrsf', 'garagearea', 'fullbath'] + [targetvar]
houseprices = houseprices[cols].dropna().reset_index(drop=True)
X = houseprices.drop(columns=[targetvar])
y = houseprices[targetvar]

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)
scaled_X = sm.add_constant(scaled_X)
scaled_X = pd.DataFrame(scaled_X,
             columns=['Const'] + list(X.columns))

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.25, random_state=36)

results = sm.OLS(y_train, X_train).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.601
Method:                 Least Squares   F-statistic:                     339.5
Date:                Sat, 30 May 2020   Prob (F-statistic):          1.01e-177
Time:                        13:29:47   Log-Likelihood:                -11000.
No. Observations:                 900   AIC:                         2.201e+04
Df Residuals:                     895   BIC:                         2.203e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Const       1.804e+05   1645.820    109.610      0.000    1.77e+05    1.84e+05
lotarea     8899.3889   2734.590      3.254      0.001    3532.432    1.43e+04
firstflrsf  2.149e+04   2085.961     10.302      0.000    1.74e+04    2.56e+04
garagearea  2.912e+04   2033.716     14.317      0.000    2.51e+04    3.31e+04
fullbath    2.263e+04   1876.511     12.059      0.000    1.89e+04    2.63e+04
==============================================================================
Omnibus:                      164.190   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3279.706
Skew:                          -0.105   Prob(JB):                         0.00
Kurtosis:                      12.350   Cond. No.                         2.48
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<span style="color:blue">There is no significant difference in the coefficients between the model with and without the insignificant features</span>

In [45]:
X.head(5)

,lotarea,firstflrsf,garagearea,fullbath
0,8450,856,548,2
1,9600,1262,460,2
2,11250,920,608,2
3,9550,961,642,1
4,14260,1145,836,2


<span style="color:blue">Based on the coefs and the scale of the features, lotarea and fullbath have the most impact on salesprice</span>